# Web Scraping

In [1]:
import requests
import csv
from bs4 import BeautifulSoup

## Traer con la librería requests la web

In [6]:
# URL de la portada de Menéame
url = "https://www.meneame.net/"

# Encabezados para simular un navegador real
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Obtener la página
response = requests.get(url, headers=headers)

In [7]:
session = requests.session()
response = session.get(url, headers=headers)
response

<Response [403]>

## Manejo de errores

In [4]:
response.raise_for_status()
# Esto fallaría si hubiese algo que no sea un 2XX
response.status_code

HTTPError: 403 Client Error: Forbidden for url: https://www.meneame.net/

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")

## Encontrar las noticias en el HTML

### Preguntas que me debo hacer

¿Qué puedo hacer para identificar qué partes son noticias?

¿Qué clases CSS utilizan las noticias?

In [ ]:
noticias = soup.find_all("div", class_="news-summary")

## Por cada noticia, recuperar el numero de meneos, clicks y comentarios

Puedo ir probando con una sola noticia a ver que sale...




In [ ]:
# Veamos lo que trae una noticia
noticias[0]

In [ ]:
# Un poco más bonito
print(noticias[0].prettify())

## ¿Cómo puedo recuperar el número de meneos?

* https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* https://www.tutorialspoint.com/beautiful_soup/beautiful_soup_navigating_by_tags.htm

Los Tag de beautiful soup tienen métodos que pueden sernos útiles
* tag.find(name, attrs, recursive, text, limit) -> Devuelve el primer child que cumpla los criterios.
* tag.find_all(name, attrs, recursive, text, limit) -> Devuelve una lista de los child que cumpla los criterios.
* tag.select(css_selector) -> Permite usar CSS para obtener elementos.
* tag.parent -> Retorna el padre de un tag.
* tag.text -> Retorna el texto interno de un tag.
* tag.html -> Retorna el HTML interno de un tag.
* tag.get_text(strip=True) -> Retorna el texto interno de un tag (quitando etiquetas internas)




In [ ]:
# Recuperar meneo
noticia = noticias[0]
meneos = noticia.find("div", class_="votes").get_text(strip=True)
print(meneos)

In [ ]:
# Recuperar meneo
noticia = noticias[0]
meneos = noticia.find("div", class_="votes").find('a').text
print(meneos)

In [ ]:
# Recuperar click
noticia = noticias[0]
clicks = noticia.find("div", class_="clics").find('span').text
print(clicks)

In [ ]:
# Recuperar num de comentarios
noticia = noticias[0]
ncoment = noticia.find("div", class_="news-details-main").find('a', class_="comments").text
print(ncoment)

## Expresiones regulares

Las expresiones regulares (regex) son patrones que permiten buscar, extraer y manipular texto de manera eficiente. Se utilizan en muchos lenguajes de programación, incluyendo Python, con la librería re que ya viene instalada.

Librería re de python
* https://docs.python.org/es/3.13/library/re.html

Recurso interesante
* https://ihateregex.io/


In [ ]:
# EXTRA: Utilizar expresiones regulares para quedarnos solo con el numero
import re

c = re.search(r'\d+', ncoment)
int(c[0])

In [ ]:
type(c)

In [ ]:
c.group()

In [ ]:
type(c.group())

In [ ]:
# Obtener la URL de la noticia
# Podemos acceder al atributo de un tag como si fuera un diccionario.

url_noticia = noticia.find("a", class_="comments")["href"]
url_noticia

In [ ]:
noticias_lista = []
for noticia in noticias:
    # Obtener el título
    titulo = noticia.find("h2").get_text(strip=True)

    # Obtener número de meneos
    meneos = meneos = noticia.find("div", class_="votes").find('a').text

    # Obtener número de clics
    clicks = noticia.find("div", class_="clics").find('span').text

    # Obtener número de comentarios
    ncoment = noticia.find("div", class_="news-details-main").find('a', class_="comments").text
    comentarios = re.search(r'\d+', ncoment).group()

    # Obtener la fuente de la noticia (dominio)
    fuente = noticia.find("span", class_="showmytitle").get_text(strip=True)

    # Obtener la URL de la noticia
    url_noticia = noticia.find("a", class_="comments")["href"]
    url_noticia = f"https://meneame.net/{url_noticia}"

    # Agregar la noticia a la lista
    noticias_lista.append([titulo, meneos, clicks, comentarios, fuente, url_noticia])



In [ ]:
# Guardar en un archivo CSV con manejo seguro de comas y caracteres especiales
import os

basepath = os.getcwd()
filename = "noticias.csv"
fullpath = os.path.join(basepath, filename)

with open(fullpath, mode="w", newline="", encoding="utf-8") as file:
    print("Guardando archivo")
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)  # Se asegura de encerrar los valores en comillas
    writer.writerow(["Título", "Meneos", "Clicks", "Comentarios", "Fuente", "URL"])  # Encabezados
    writer.writerows(noticias_lista)
print(f"Archivo guardado en {filename}.")
